In [2]:
from pyspark import SparkConf, SparkContext
import collections
import pandas as pd

# spark = SparkSession.builder().master("local[2]")
#           .appName("Movie Review Exploration")
#           .getOrCreate()
# df = spark.read.csv("/ml-latest-100k/reviews.csv")
conf = SparkConf().setMaster("local").setAppName("RatingsHistogram")
sc = SparkContext(conf = conf)

22/09/18 09:17:53 WARN Utils: Your hostname, Zipcoders-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.239 instead (on interface en0)
22/09/18 09:17:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/18 09:17:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
lines = sc.textFile("ml-100k/u.data")
ratings = lines.map(lambda x: x.split()[2])
result = ratings.countByValue()

In [3]:
sortedResults = collections.OrderedDict(sorted(result.items()))
for key, value in sortedResults.items():
    print(f"{key}: {value}")

1: 6110
2: 11370
3: 27145
4: 34174
5: 21201


In [3]:
# some Pandas for exploring the data. Comparisons will be made via Spark later.

df = pd.read_csv("./ml-latest-small/movies.csv")
df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
import os

csvs = os.listdir("./ml-latest-small")
csvs = [file for file in csvs if '.csv' in file]
csvs

['links.csv', 'tags.csv', 'ratings.csv', 'movies.csv']

In [9]:
# Links: 
# | index | movieID \ IMDB movie # | MovieDB movie # |
links_df = pd.read_csv("./ml-latest-small/" + csvs[0])
links_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [7]:
# Tags
# | index | userID \ movieID \ tag \ timestamp
# timestamp is: seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970
tags_df = pd.read_csv("./ml-latest-small/" + csvs[1])
tags_df.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [10]:
# Ratings
# | index | userID \ movieID \ Rating (0-5 w/ 0.5 increments) | timestamp
# timestamps are: seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970
ratings_df = pd.read_csv("./ml-latest-small/" + csvs[2])
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [11]:
# Movies
# | index | movieID \ Title \ Genrespd.
movies_df = pd.read_csv("./ml-latest-small/" + csvs[3])
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [20]:
combined_df = pd.merge (tags_df, ratings_df, on="movieId")
combined_df.loc[combined_df['movieId'] == 60756]

,userId_x,movieId,tag,timestamp_x,userId_y,rating,timestamp_y
0,2,60756,funny,1445714994,2,5.0,1445714980
1,2,60756,funny,1445714994,18,3.0,1455749449
2,2,60756,funny,1445714994,62,3.5,1528934376
3,2,60756,funny,1445714994,68,2.5,1269123243
4,2,60756,funny,1445714994,73,4.5,1464196221
...,...,...,...,...,...,...,...
219,424,60756,will ferrell,1457846129,484,4.5,1342368380
220,424,60756,will ferrell,1457846129,560,3.5,1469653998
221,424,60756,will ferrell,1457846129,564,5.0,1478451501
222,424,60756,will ferrell,1457846129,583,3.0,1481474191


In [ ]:
# The earlier set had user data as well. May use that instead for additional information for recommendations.